# Memory
- 기본 개요
    - 랭체인에는 5가지 정도의 저장방식이 있음
    - 각자 사용방법/ 장단점이 다름
    - 챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없음
    - 유저가 자신의 이름을 말하거나 이전 질문에 이어지는 질문을 해도 메모리, 즉 기역력이 없기때문에 대화를 이해할 수 있는 능력이 없음
    - openAI에서 제공하는 기본 API는 랭체인없이 사용할 수 있지만 메모리를지원하지 않음
    - 즉 어떤 모델에 대화를 걸어더 나중에 다 까먹게됨
    - 챗gpt에는 메모리가 존재하고 실제로 어떤 사람과 대화하는 느낌을 가지게함
    - 챗봇이 이전의 대화나 질문을 기억하고 답할 수 있기때문임
    - 따라서 이번 강의에서는 이런 각 메모리의 종류와 차이점을 보고, 랭체인에 메모리를 탑재하는 방법을 알아봄
- 기본 흐름
    - 모든 메모리는 save_context, load_memory_variables, input, output 함수를 가지고 있음, 메모리 종류만 다를뿐 API는 동일함
- return_message
    - 메모리를 우선적으로 만들고, 챗 모델을 위한것 인지 아닌지 선택하고, 챗모델을 위한게 아니라면 return_message=False로 하거나 빼도 됨
    - 그럼 history가 문자열로만 표시될 것
    - 쳇모델을 사용하고싶다면 True로 바꾸면 챗 모델이 사용할 수 있는 형태로 출력됨


## 5.1 Conversation Buffer Memory
- 단순히 이전 대화내용을 기억하는 메모리
- 장점 : 가장 이해하기 쉬운 메모리임
- 단점 : 대화내용이 길어질수록 메모리도 커지니까 비효율적임


- 모델 자체에는 메모리가 없음
  - 우리가 모델에 요청을 보낼때 이전 대화 기록 전체를 같이 보내줘야함
  - 그래야 모델이 전에 일어난 대화를 보고 이해할 수 있음
  - 유저와 AI 대화가 길어질 수 록 우리가 매번 모델에 보내야할 기록이 길어진것을 의미하며 상당히 비효율적이고 돈이 많이 듬



In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({}) # history로 대화한 내용을 불러움



{'history': 'Human: Hi\nAI: 뭐하니?'}

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({}) # history로 대화한 내용을 불러움
# {'history': [HumanMessage(content='Hi'), AIMessage(content='뭐하니?')]}

{'history': [HumanMessage(content='Hi'), AIMessage(content='뭐하니?')]}

In [4]:
# 이 메모리는 대화를 저장하고 불러오는 기능을 가지기 떄문에 메모리에 대화내용이 계속 누적시킴

memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?')]}

In [5]:
memory.save_context({"input":"Hi"}, {"output":"뭐하니?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?'),
  HumanMessage(content='Hi'),
  AIMessage(content='뭐하니?')]}

In [ ]:
# 대화가 길어질 수 록 수많은 내용이 계속 쌓이기때문에 비효율적이라, 좀 더 효율적으로 메모리, 대화 기록을 최소화하는 메모리를 확인할 것


## 5.2 ConversationBufferWindowMemory
- k=4 처럼 메모리 개수를 지정하여 모든 대화를 저장하지 않고 일부만 저장하도록 함
- 최신 것만 기억하는 것이 단점이지만 비용이 효율적일 수 있음

In [11]:
from langchain.memory import ConversationBufferWindowMemory

# 대화의 특정 부분만을 저장하는 메모리
# 예: 최근 5개만 저장하는 경우, 최근 5개만 저장하고 나머지는 삭제됨
# 메모리를 특정 크기로 유지할 수 있고, 모든 대화를 저장하지 않아도 됨
# 단점은 챗봇이 최근 대화만 기억함

memory = ConversationBufferWindowMemory(
    return_messages=True, 
    k = 4
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    

add_message(1,1)

In [12]:
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

In [13]:
memory.load_memory_variables({})
# k=4이기 때문에 1이 삭제됨/ 최신 4개만 저장됨


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [14]:
add_message(6,6)
memory.load_memory_variables({}) # 최신 4개만 저장되어 3~6까지만 저장/ 단점은 챗봇이 최근 대화만 기억함

{'history': [HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}

## 5.3-1 ConverstaionSummaryMemory
- 메세지를 그대로 저장하는것이 아니라 conversation 자체를 요약하는것
- llm을 필요로함
- 매우 긴 conversation이 있는 경우 유용함
- 초반에는 이전보다 많은 토큰고 저장공간을 차지하지만 메세지가 많아질수록 도움이 됨
- 요약하는 것이 토큰 양도 줄면서 훨씬 나은 방법이됨
- 저장하는 것보다 COnversation을 사용하여 요약하는것이 훨씬 나음


In [15]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryMemory(llm = llm) 

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})

add_message("안녕 여기는 서울입니다.","서울의 날씨는 어떠니?")


In [16]:
add_message("여기는 광주입니다.","광주가고싶다")

In [18]:
print(get_history())
""" 문장을 요약해서 저장해둠/ 내가 입력한 것 보다는 처음에 더 맣은 저장공간을 사용하여 토큰을 사용하지만 대화가 길어질 수록 summary가 각 메세지보다 더 효율적임
{'history': 'The human greets in Korean and mentions they are in Seoul. The AI asks about the weather in Seoul. The human responds that they are in Gwangju. The AI expresses a desire to visit Gwangju.'}
"""

{'history': 'The human greets in Korean and mentions they are in Seoul. The AI asks about the weather in Seoul. The human responds that they are in Gwangju. The AI expresses a desire to visit Gwangju.'}


## 5.3-2 ConversationSummaryBufferMemory
- Conversation Buffer Memory와 Conversation Summary Memory의 결합임
- 메모리에 보내온 메세지 수를 제한
- 오래된 메세지들을 요약함
- 가장 최근의 상호작용을 계속 추적하며
- 가장 최근/ 가장 오래전에 받은 이야기를 모두 추적하고있다는 것

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_tokens = 30, # 최대 토큰 수/ 메세지들이 요약되기 전까지의 토큰 수 
    return_messages=True # 채팅모델에 좋음. 
) 

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})

def get_history():
    return memory.load_memory_variables({})


In [ ]:
add_message("온톨로지에 대해 설명해줘.","온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야")
get_history()

# max tokens 값이이 30이기 때문에 30을 넘어가면 요약이 되어 저장됨. 아래처럼 SystemMessage로 저장되어 질문과 답변에 대해 요약함
"""
'history': [SystemMessage(content='The human asks the AI to explain ontology. The AI explains that ontology is a knowledge structure design that expresses the concepts (objects) of reality and the relationships between concepts to understand the meaning of data.'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
"""

# 최종적으로 system이 내용을 요약하고 저장하고 있으므로, 이건 실제 요금이 청구되는 API를 사용하고 있다는걸 의미함
# 따라서, 최대한 효율적으로 사용하는 것이 좋음

{'history': [SystemMessage(content='The human asks the AI to explain ontology. The AI explains that ontology is a knowledge structure design that expresses the concepts (objects) of reality and the relationships between concepts to understand the meaning of data.'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를 파악하게 하는 지식의 구조 설계야'),
  HumanMessage(content='온톨로지에 대해 설명해줘.'),
  AIMessage(content='온톨로지는 현실의 개념(사물)과 개념간의 관계를 표현하여 데이터의 의미를

### 5.4 ConversationKGMemory
- LLM을 사용하는 메모리 클래스
- 대화중의 엔티티의 지식 그래프를 생성함
- 가장 중요한 것들만 뽑아내는 요약본같은 것
- 지식그래프에서 히스토리를 가져오지 않고 엔티티를 가져옴

- 참고 : https://python.langchain.com/v0.1/docs/modules/memory/types/kg/

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

# chat= ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
llm= ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm = llm,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input":input}, {"output":output})
    v
add_message("say hi to sam", "who is sam")
add_message("sam is a friend", "okay")

# 출력
memory.load_memory_variables({"input": "who is sam"})

{'history': [SystemMessage(content='On sam: sam is a friend.')]}

In [45]:
memory.get_knowledge_triplets("sam's favorite color is red")

[KnowledgeTriple(subject='sam', predicate='favorite color is', object_='red')]

In [54]:

from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

llm = ChatOpenAI(temperature=0)

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm, verbose=True, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


'Hello! I am currently running on a neural network with deep learning capabilities. My primary function is to assist users with information and tasks based on the data provided to me. How can I help you today?'

In [50]:
memory.get_knowledge_triplets("sam's favorite color is red")

[KnowledgeTriple(subject='sam', predicate='favorite color is', object_='red')]

In [55]:
conversation_with_kg.predict(
    input="My name is James and I'm helping Will. He's an engineer."
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


"Hello James! It's nice to meet you. How can I assist you with helping Will, the engineer?"

In [56]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is an engineer.

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


'Will is an engineer. He likely has a strong background in math and science, as well as problem-solving skills. He probably works with technology and machinery on a regular basis. Will may also have a detail-oriented and analytical mindset, which are important qualities for an engineer.'

## 5.5 Memory on LLMChain
- 메모리를 chain에 꽂는 방법과 두 종류의 chain을 사용해서 꽂는 방법을 배움
- llm chain : off the shelf는 일반적인 목적을 가진 체인을 의미함
    - 랭체인에 아주 많고 유용함
    - 하지만 우리가 스스로 무언가를 만들어볼때 off the shelf chain 보다는 직접 커스텀해서 만든 체인을 만들어서 활용하는 것을 추천
    - 빠르게 시작할수있어 좋지만
    - 프레임워크를 다루느라 고생하거나 직접 커스텀하고싶을때 이전에 배운것처럼 langchain expression언어를 활용해서 우리의 것을 만들 수 있음


    




In [18]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# 메모리에는 ConversationSummaryBufferMemory를 사용하고 있음
# llmchain에 이 메모리를 넣는 것을 배움
# 이게 interaction을 기본적으로 가지고 있는 conversation chain임
# 그리고 interaction 토큰 수가 max값보다 크면 가장 오래된 interactiond을 요약해주고 최신내용은 유지해
# 그냥 ConversationBufferMemory를 사용하는 것도 상관없음음


llm = ChatOpenAI(
model_name="gpt-4o-mini",
temperature=0.1,
tiktoken_model_name="gpt-3.5-turbo",
)
# llm= ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)
# llm= ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, # 80 이상이면 요약해줌-> 120 변경
    memory_key  = "chat_history", # 메모리에 저장할 키값/메모리는 chat_history라는 키값을 template에서 찾아서 관련 내용을 찾아줌
)

# chain = LLMChain(
#     llm = llm,
#     memory = memory,
#     prompt =PromptTemplate.from_template("{question}"), # LLM에게 question만 주고있음
#     verbose = True # 프롬프트 답변을 볼수있는 옵션션
# )

# hisotry를 알수있게함 - chat_history 이름을 바꿔도 되고, 템플릿에 이전 기록에 뭔가 작업을 하라고 해도 되고 커스텀하면됌. 대신 memory_key와 이름을 맞춰야함함
template = """

    You are a helpful AI talking to a human.
    {chat_history} 
    Human: {question}   
    You : 
"""

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt =PromptTemplate.from_template(template), # LLM에게 question만 주고있음
    verbose = True # 프롬프트 답변을 볼수있는 옵션션
)

chain.predict(question="My Name is Chloe")






> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.
     
    Human: My Name is Chloe   
    You : 


> Finished chain.


"Hello, Chloe! It's nice to meet you. How can I assist you today?"

In [24]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.
    System: The human introduces herself as Chloe and mentions that she lives in Seoul. The AI greets her and expresses interest in her experience, asking what she enjoys most about living in the vibrant city.
Human: What is my name?
AI: Your name is Chloe!
Human: I live in Seoul
AI: That's wonderful, Chloe! Seoul has so much to offer. Do you have any favorite places or activities in the city?
Human: I live in Seoul
AI: AI: I see you really love to emphasize that! Seoul is indeed a fascinating place. Is there something specific about living in Seoul that you'd like to share or discuss? 
    Human: I live in Seoul   
    You : 


> Finished chain.


'AI: It sounds like Seoul is really important to you! What aspects of life in Seoul do you find most enjoyable or unique?'

In [25]:
chain.predict(question="What is my name?") # 대답할수없음답할수없음
# 우리가 대화내역을 말해준적이 없어서 기억이 기록되지 않음

# 마지막으로 token이 120을 넘어가면 요약해주는 것도 잘 동작함 System



> Entering new LLMChain chain...
Prompt after formatting:


    You are a helpful AI talking to a human.
    System: The human introduces herself as Chloe and mentions that she lives in Seoul. The AI greets her and expresses interest in her experience, asking what she enjoys most about living in the vibrant city. The human confirms her name is Chloe and reiterates that she lives in Seoul.
AI: That's wonderful, Chloe! Seoul has so much to offer. Do you have any favorite places or activities in the city?
Human: I live in Seoul
AI: AI: I see you really love to emphasize that! Seoul is indeed a fascinating place. Is there something specific about living in Seoul that you'd like to share or discuss?
Human: I live in Seoul
AI: AI: It sounds like Seoul is really important to you! What aspects of life in Seoul do you find most enjoyable or unique? 
    Human: What is my name?   
    You : 


> Finished chain.


"Your name is Chloe! It's great to chat with you. How has your experience been living in Seoul?"

In [21]:
memory.load_memory_variables({}) # 대화내역을 불러와보기보기

{'chat_history': "Human: My Name is Chloe\nAI: Hello, Chloe! It's nice to meet you. How can I assist you today?\nHuman: I live in Seoul\nAI: That's great, Chloe! Seoul is a vibrant city with a rich culture and history. What do you enjoy most about living there?\nHuman: What is my name?\nAI: Your name is Chloe!"}

### 5.6. Chat Based Memory
- 위의 결과는 그저 텍스티일뿐임. 이걸 대화기반의 채팅으로 바꾸는 방식
- 대화 기반 메세지 메모리는 매우 쉬움
- 문자열 형태 또는 메세지 형태일 수 있음
예를들어, 메모리쪽에 저렇게 보면 기록된 프롬프트 방식을 가져올수있는데 채팅기반으로 바꾸려면 문자열이 아니라 메세지로 바꿔달라고 하면됨
- 문자열 기반 템플릿대신 chatPromptTemplate을 가져오게 하면됨


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [28]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate,MessagesPlaceholder
# ConversationSummaryBufferMemory가 시스템요약, 사람, ai, 사람, ai 등의 메세지에 기록해나갈때 우리는 그게 얼마나 많은지 알수없기에 MessagesPlaceholder를 사용함
# MessagesPlaceholder 역할은 우리가 메세지가 얼마나 많고 누구에게 왔는진 모르지만 메세지 클래스로 대체될 거임

llm = ChatOpenAI(
model_name="gpt-4o-mini",
temperature=0.1,
tiktoken_model_name="gpt-3.5-turbo",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, # 80 이상이면 요약해줌-> 120 변경
    memory_key  = "chat_history", # 메모리에 저장할 키값/메모리는 chat_history라는 키값을 template에서 찾아서 관련 내용을 찾아줌
    return_messages=True,
)

Prompt= ChatPromptTemplate.from_messages([
    ("system", "너는 사람에게 도움을 주는 AI야"),
    MessagesPlaceholder(variable_name="chat_history"), # 변수명을 넣어줌 -> 대화요약 메모리는 그 기록을 가져와서 massagePlaceholder에 넣어줌
    ("human", "{question}")
    
])
# MessagesPlaceholder : 사람, ai, 사람, ai 등등 반복될 것들을 넣어줌, 누가보냈는지 알수없는 예츠갛기 어려운 메세지의 양과 제한없는 양의 메세지에 대해 시스템인지 사람, Ai 메세지인지 알수없음음

chain = LLMChain(
    llm = llm,
    memory = memory,
    prompt =Prompt,
    verbose = True # 프롬프트 답변을 볼수있는 옵션션
)


chain.predict(question="My Name is Chloe")

# 결과 해석하기
# system 나오고 Human이 순서대로 나옴, 그 다음 AI - Human 순으로 반복됨




> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람에게 도움을 주는 AI야
Human: My Name is Chloe

> Finished chain.


'Nice to meet you, Chloe! How can I assist you today?'

In [33]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람에게 도움을 주는 AI야
System: The human introduces herself as Chloe, and the AI responds by expressing pleasure in meeting her and asking how it can assist her today. Chloe mentions that she lives in Seoul, and the AI acknowledges this by highlighting Seoul's vibrant culture and history, asking if there's something specific she would like to know or discuss about the city.
Human: what is my name?
AI: Your name is Chloe!
Human: I live in Seoul
AI: Yes, you mentioned that you live in Seoul. How do you like living there? Is there anything specific you want to discuss about the city?
Human: I live in Seoul
AI: Got it! If there's anything specific you'd like to share about living in Seoul or if you have any questions or topics in mind, feel free to let me know!
Human: I live in Seoul

> Finished chain.


"I understand! Living in Seoul must be an exciting experience. If there's anything specific you'd like to talk about regarding your life there, such as places to visit, food to try, or cultural experiences, just let me know!"

In [34]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람에게 도움을 주는 AI야
System: The human introduces herself as Chloe, and the AI expresses pleasure in meeting her and asks how it can assist her today. Chloe mentions that she lives in Seoul, and the AI acknowledges this by highlighting Seoul's vibrant culture and history, asking if there's something specific she would like to know or discuss about the city. Chloe then asks the AI what her name is, to which the AI responds that her name is Chloe. The AI reiterates her residence in Seoul and inquires how she likes living there and if there's anything specific she wants to discuss about the city.
Human: I live in Seoul
AI: Got it! If there's anything specific you'd like to share about living in Seoul or if you have any questions or topics in mind, feel free to let me know!
Human: I live in Seoul
AI: I understand! Living in Seoul must be an exciting experience. If there's anything specific you'd like to talk about regarding 

"Your name is Chloe! It's nice to meet you, Chloe. How can I assist you today?"

In [35]:
memory.load_memory_variables({}) # 대화내역을 불러와보기보기

{'chat_history': [SystemMessage(content="The human introduces herself as Chloe, and the AI expresses pleasure in meeting her and asks how it can assist her today. Chloe mentions that she lives in Seoul, and the AI acknowledges this by highlighting Seoul's vibrant culture and history, asking if there's something specific she would like to know or discuss about the city. Chloe then asks the AI what her name is, to which the AI responds that her name is Chloe. The AI reiterates her residence in Seoul and inquires how she likes living there and if there's anything specific she wants to discuss about the city. Chloe confirms her residence in Seoul, and the AI encourages her to share more about her experiences or any questions she may have."),
  HumanMessage(content='I live in Seoul'),
  AIMessage(content="I understand! Living in Seoul must be an exciting experience. If there's anything specific you'd like to talk about regarding your life there, such as places to visit, food to try, or cult

### 5.7 LCEL Based Memory
- langchain expression 언어를 이용해서 생성된 체인에 메모리를 추가하는 것은 어렵지 않고실제로 변경작업을 할때 권장되는 방법
- 메모리를 추가하는게 LLM체인을 사용하는 것만큼 쉬움



In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
    tiktoken_model_name="gpt-3.5-turbo",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history", 
    return_messages=True,
)

Prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 사람에게 도움을 주는 AI야"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

# 방법1 : invoke에서 메모리를 직접 넣어주는 방식식
chain = Prompt | llm

chain.invoke({
    "chat_history" : memory.load_memory_variables({})["chat_history"],
    "question" : "My Name is Chloe"
})


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
    tiktoken_model_name="gpt-3.5-turbo",
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history", # 기본적으로 메모리키가 history로 설정되어 있어 꼭 chat_history로 안써도 가능(아래 key 명칭을 다 history로 바꾸면 됨됨)
    return_messages=True,
)

Prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 사람에게 도움을 주는 AI야"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)



# 방법2
# def load_memory(input): # chain에선 모든 것이 input을 얻고, 그 후 output을 주는 규칙
#     print("input = ", input)
#     return memory.load_memory_variables({})["history"]


def load_memory(_):  # 명시적으로 input이라 하지않아도 _ 해도 문제없음음
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | Prompt | llm
# RunnablePassthrough 먼저 실행되며 Prompt에게 history를 전달하고, Prompt는 llm에게 전달함
# assign은 변수를 할당하는 것
# 해당 함수에서 원하는 어떤 값이던 변수에 할당할 수 있음


chain.invoke(
    {
        "question": "My Name is Chloe"  # 우리가 입력한 질문이 chain에서 첫번째 아이템의 input으로 입력됨(RunnablePassthrough~ 이쪽) : 규칙임
    }
)

"""
체인에는 메모리가 존재하고, 우리는 load_memory라는 함수를 실행하고, 해당 함수는 history의 키를 사용하는 prompt에 전달되어야함
=> load_memory를 실행시키고, 그 함수를 실행시키고 얻은 output을 history라는 속성으로 들어가야함
그리고 그게 사용자의 input과 결합되어 프롬프트에 들어갈거고, 프롬프트는 history와 질문이 필요함

"""

# 우리는 메모리 관리를 수동으로하고있어서 모든 대화가 저장될 것임
# 코드를 통제하기 위해 코드를 좀 더 드러내서 작성하는 방식으로 진행함함
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [49]:
invoke_chain("My name is Chloe")

content='Nice to meet you, Chloe! How can I assist you today?'


In [50]:
invoke_chain("What is my name?")

content='Your name is Chloe. How can I help you today, Chloe?'
